Experiment based on article: https://ieeexplore.ieee.org/document/10078518
Dataset: https://slobodan.ucg.ac.me/science/vs13/
Kaggle notebook: https://www.kaggle.com/code/markbarreto/vehicle-speed 

**Install Ultralytics and Supervision**

In [57]:
!pip install -q ultralytics
!pip install -q supervision==0.11.1

In [58]:
import os
import cv2
import glob
import torch
import numpy as np
import supervision as sv
from PIL import Image
from tqdm import tqdm
from torch import optim
from ultralytics import YOLO
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error as mse

In [59]:
def area(bbox):
    length = bbox[3] - bbox[1] 
    width = bbox[2] - bbox[0]
    return float(length * width)

def linear_interpolation(areas):
    first_frame = areas[0][0]
    last_frame = areas[-1][0]
    if last_frame - first_frame > len(areas):
        new_areas = []
        count_frame = first_frame
        for idx, frame in enumerate(areas):
            if frame[0] == count_frame:
                new_areas.append(frame)
                continue
            x = count_frame
            x1 = count_frame - 1
            x2 = frame[0]
            fraction = (x - x1) / (x2 - x1)
            interpolated_area = areas[idx - 1][1] + (frame[1] - areas[idx - 1][1]) * fraction
            new_areas.append((count_frame, interpolated_area))
            count_frame += 1
        return new_areas
    return areas

def lne_interpolation(areas, length):
    if len(areas) >= length:
        return areas
    last_point = areas.pop()
    count_frame = 0
    for idx in range(len(areas), length - 1):
            count_frame = areas[idx - 1][0] + 1
            x = count_frame
            x1 = areas[idx - 1][0]
            x2 = count_frame + 1
            fraction = (x - x1) / (x2 - x1)
            interpolated_area = areas[idx - 1][1] + (last_point[1] - areas[idx - 1][1]) * fraction
            areas.append((count_frame, interpolated_area))
    point = (count_frame + 1, last_point[1])
    areas.append(point)
    return areas
    
def aux_interpolation(total_areas, save = False, root = '/kaggle/working'):
    length = 0
    if not os.path.isdir(root):
        os.mkdir(root)
    for folder, values in total_areas.items():
        for key, areas in values.items():
            teste = linear_interpolation(areas)
            values[key] = teste
            aux = len(teste)
            if aux > length:
                length = aux
                
    for folder, values in total_areas.items():
        for key, areas in values.items():
            areas[key] = lne_interpolation(areas, length)
            if save:
                file = os.path.join(root, f'{folder}.txt')
                with open(file, 'a+') as f:
                    for area in areas:
                        f.write(str(area[1]) + ' ')
                    f.write('\n')
        
    return total_areas

In [60]:
def get_areas(model, files):
    total_areas = {}
    classes = [2, 3, 5, 7]
    for file in files:
        for frame_idx, result in enumerate(model.track(source=file, show = False, stream=True)):
            detections = sv.Detections.from_yolov8(result)
            
            if result.boxes.id is not None:
                detections.tracker_id = result.boxes.id.cpu().numpy().astype(int)

            detections = detections[np.isin(detections.class_id, classes)]
            
            if detections.tracker_id is not None:
                for idx in range(len(detections.tracker_id)):
                    tracker_id = detections.tracker_id[idx]
                    
                    if tracker_id in cbba_file.keys():
                        cbba_file[tracker_id].append((frame_idx, area(detections.xyxy[idx])))
                    else:
                        cbba_file[tracker_id] = [(frame_idx, area(detections.xyxy[idx]))]
            
        key = file.split('/')[-1].replace('.MP4', '')
        total_areas[key] = cbba_file
    return total_areas

**Creating dataset with areas of the bounding boxes**

In [ ]:
HOME = '/kaggle/input/vs13-dataset/vs13'

FOLDERS = [
    'CitroenC4Picasso',
    'KiaSportage',
    'Mazda3',
    'MercedesAMG550',
    'NissanQashqai',
    'OpelInsignia',
    'Peugeot208',
    'Peugeot307',
    'Peugeot3008',
    'RenaultCaptur',
    'RenaultScenic',
    'VWPassat'
]

output = '/kaggle/working/data'
file_areas = {}
for folder in FOLDERS:
    files = glob.glob(os.path.join(HOME, folder, '*.MP4'))
    model = YOLO('yolov8n.pt')
    total_areas = get_areas(model, files)
    aux_interpolation(total_areas, save = True, root = output)
    file_areas[folder] = total_areas

In [61]:
def get_normalize(dataset):
    max_cbba = 0
    for data in dataset:
        for value in data[1]:
            if value > max_cbba:
                max_cbba = value
    return max_cbba

class AreaDS(Dataset):
    def __init__(self, dataset, max_cbba):
        self.dataset = dataset
        self.max_cbba = max_cbba
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        label = self.dataset[idx][0].split('_')[-1]
        text = torch.tensor([self.dataset[idx][1]], dtype = torch.float64)
        text = torch.div(text, self.max_cbba)
        label = torch.tensor(float(label), dtype=torch.float64)
        return text, label

In [62]:
def train_test_split(path):
    folders = glob.glob(path)

    train_areas = []
    valid_areas = []
    for idx in range(1, len(folders)):
        files = glob.glob(f'{folders[idx]}/*.txt')
        shuffle = np.random.permutation(len(files))
        count_files = 0
        valid_size = int(3*int(len(files) / 10))
        for file in files:
            with open(file, 'r') as f:
                key = file.split('/')[-1].replace('.txt', '')
                line = f.readline()
                aux = line.split(' ')[:-1]
            float_area = [float(area) for area in aux]
            if count_files < valid_size:
                valid_areas.append([key, float_area])
            else:
                train_areas.append([key, float_area])
            count_files += 1
    return train_areas, valid_areas

def aux_balance(data, length):
    if length == len(data):
        return data
    for idx in range(length, len(data)):
        data.pop()
    return data

def balance_data(dataset):
    length_max = -1
    length_min = 99999999999999999
    for data in dataset:
        length = len(data[1])
        if length > length_max:
            length_max = length
        if length < length_min:
            length_min = length
    if length_min != length_max:
        for data in dataset:
            if len(data[1]) != length_min:
                data[1] = aux_balance(data[1], length_min)
    return dataset



In [63]:
train_areas, valid_areas = train_test_split('/kaggle/input/vs13-areas/areas/*')
#train_areas = balance_data(train_areas)
#valid_areas = balance_data(valid_areas)
dataset = train_areas + valid_areas
max_cbba = get_normalize(dataset)
train_data = AreaDS(train_areas, max_cbba)
valid_data = AreaDS(valid_areas, max_cbba)
train_loader = DataLoader(train_data, batch_size = 1, shuffle = True)
valid_loader = DataLoader(valid_data, batch_size = 1, shuffle = True)

In [64]:
class SpeedEstimator(torch.nn.Module):
    def __init__(self, inpt, output, kernel_size = 10):
        super().__init__()
        self.conv1 = torch.nn.Conv1d(inpt,output, kernel_size)
        self.pool = torch.nn.MaxPool1d(2)
        self.relu = torch.nn.ReLU(inplace = True)
        self.conv2 = torch.nn.Conv1d(output, output, kernel_size)
        self.max = torch.nn.AdaptiveMaxPool1d(output)
        self.fc = torch.nn.Linear(output**2, 1)
        self.loss = torch.nn.MSELoss()
        self.optim = optim.Adam(self.parameters())
        
    def forward(self, x):
        out = self.conv1(x.to(self.conv1.weight.dtype))
        out = self.relu(out)
        out = self.pool(out)
        
        out = self.conv2(out)
        out = self.relu(out)
        out = self.pool(out)
        
        out = self.max(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out.squeeze()
    
    def trainn(self, train_data, val_data, epochs):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.to(device)
        losses = []
        best_rmse = 9999999999
        for epoch in range(epochs):
            epoch_loss = 0.0 
            self.train()
            loop = tqdm(enumerate(train_data), total = len(train_data))
            for batch_idx, (data, target) in loop:
                data = data.to(device)
                target = target.to(device)
                
                scores = self.forward(data)
                loss = torch.sqrt(self.loss(scores, target.float()))
                
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()
                
                epoch_loss += loss.item()
            
                loop.set_postfix(epoch=epoch, epochs=epochs, train_data_len=len(train_data))
                            
            epoch_loss = epoch_loss / len(train_data)
            losses.append(epoch_loss)
            
            rmse = self.validation(val_data, device)
            if rmse < best_rmse:
                torch.save({
                    'model_state_dict': self.state_dict(),
                    'optim_state_dict': self.optim.state_dict()
                    }, 'best_model.pt'
                )
                
            print(f'loss: {epoch_loss:.2f}. rmse_eval: {rmse:.2f}')
        return best_rmse, losses
    
    def validation(self, dataset, device):
        self.eval()
        with torch.no_grad():
            rmse = 0.0
            for (data, target) in dataset:
                data = data.to(device)
                target = target.to(device)
                
                scores = self.forward(data)
                rmse += torch.sqrt(self.loss(scores, target))
            
            rmse = rmse / len(dataset)
            return rmse
    
    def load(self, path):
        checkpoint = torch.load(path)
        self.load_state_dict(checkpoint['model_state_dict'])
        self.optim.load_state_dict(checkpoint['optim_state_dict'])
        
    def predict(self, dataset):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.to(device)
        self.eval()
        with torch.no_grad():          
            results = []
            for data in dataset:
                data = torch.tensor([data], dtype = torch.float64)
                data = data.unsqueeze(dim = 0)
                data = data.to(device)

                result = self.forward(data)
                results.append(result)
        return results

In [ ]:
speed_estimator = SpeedEstimator(1, 64, 10)
best_rmse, loss = speed_estimator.trainn(train_loader, valid_loader, 1000)

100%|██████████| 251/251 [00:01<00:00, 190.63it/s, epoch=0, epochs=1000, train_data_len=251]


loss: 20.09. rmse_eval: 7.91


100%|██████████| 251/251 [00:01<00:00, 197.22it/s, epoch=1, epochs=1000, train_data_len=251]


loss: 9.00. rmse_eval: 11.83


100%|██████████| 251/251 [00:01<00:00, 189.31it/s, epoch=2, epochs=1000, train_data_len=251]


...

100%|██████████| 251/251 [00:01<00:00, 195.87it/s, epoch=709, epochs=1000, train_data_len=251]


loss: 1.48. rmse_eval: 2.87


100%|██████████| 251/251 [00:01<00:00, 195.83it/s, epoch=710, epochs=1000, train_data_len=251]


loss: 1.43. rmse_eval: 2.66


 50%|█████     | 126/251 [00:00<00:00, 187.33it/s, epoch=711, epochs=1000, train_data_len=251]

In [ ]:
speed_estimator.load('/kaggle/working/best_model.pt')

model = YOLO('yolov8n.pt')

classes = [2, 3, 5, 7]

folder = '/kaggle/input/vs13-dataset/vs13/CitroenC4Picasso'
files = glob.glob(f'{folder}/*.MP4')

vehicles_speed = {}

for file in files:
    file_areas = {}
    file_speed = {}
    for frame_idx, result in enumerate(model.track(source = file, show = False, stream = True)):
        detections = sv.Detections.from_yolov8(result)

        if result.boxes.id is not None:
            detections.tracker_id = result.boxes.id.cpu().numpy().astype(int)

        detections = detections[np.isin(detections.class_id, classes)]

        if detections.tracker_id is not None:
            for idx in range(len(detections.tracker_id)):
                tracker_id = detections.tracker_id[idx]
                class_id = detections.class_id[idx]

                if tracker_id in file_areas.keys():
                    file_areas[tracker_id].append((frame_idx, area(detections.xyxy[idx])))
                else:
                    file_areas[tracker_id] = [(frame_idx, area(detections.xyxy[idx]))]
    
    for key in file_areas.keys():
        areas = linear_interpolation(file_areas[key])
        value = [(y / max_cbba) for (x, y) in areas]
        file_speed[key] = speed_estimator.predict([value])[0]
    vehicles_speed[file] = file_speed   

In [68]:
ground_truth = []
predict = []
for key, items in vehicles_speed.items():
    aux = key.split('/')[-1].replace('.MP4', '').split('_')
    speed = [float(value) for value in items.values()]
    vehicle = aux[0]
    ground_truth.append(float(aux[1]))
    predict.append(speed)
    error = abs(100 - (speed[0]*100 / int(ground_truth[-1])))
    print(f'Vehicle: {vehicle}. Ground_truth: {ground_truth[-1]}km/h Predict: {speed[0]:.2f}km/h. Error: {error:.2f}%')
    
ground_truth = np.array(ground_truth, dtype = np.float32)
predict = np.array(predict, dtype = np.float32)
rmse = np.sqrt(mse(ground_truth, predict))
print(f'RMSE: {abs(rmse)}km/h')

Vehicle: CitroenC4Picasso. Ground_truth: 83.0km/h Predict: 82.21km/h. Error: 0.95%
Vehicle: CitroenC4Picasso. Ground_truth: 51.0km/h Predict: 51.56km/h. Error: 1.10%
Vehicle: CitroenC4Picasso. Ground_truth: 54.0km/h Predict: 49.92km/h. Error: 7.55%
Vehicle: CitroenC4Picasso. Ground_truth: 96.0km/h Predict: 96.68km/h. Error: 0.71%
Vehicle: CitroenC4Picasso. Ground_truth: 72.0km/h Predict: 69.67km/h. Error: 3.24%
Vehicle: CitroenC4Picasso. Ground_truth: 44.0km/h Predict: 43.72km/h. Error: 0.64%
Vehicle: CitroenC4Picasso. Ground_truth: 92.0km/h Predict: 90.58km/h. Error: 1.55%
Vehicle: CitroenC4Picasso. Ground_truth: 48.0km/h Predict: 47.26km/h. Error: 1.54%
Vehicle: CitroenC4Picasso. Ground_truth: 38.0km/h Predict: 35.38km/h. Error: 6.90%
Vehicle: CitroenC4Picasso. Ground_truth: 63.0km/h Predict: 53.43km/h. Error: 15.18%
Vehicle: CitroenC4Picasso. Ground_truth: 87.0km/h Predict: 87.74km/h. Error: 0.85%
Vehicle: CitroenC4Picasso. Ground_truth: 80.0km/h Predict: 79.17km/h. Error: 1.04%
Veh